In [5]:
import sys
sys.path.append(
    '/home/macrodrigues/code/macrodrigues/my_projects/butterfly_detector/')
from scripts.transfer_learning import PATH, PATH_TEST
from scripts.transfer_learning import imgs_gen, train_gen, val_gen
from scripts.transfer_learning import compile_model, train_model 
from scripts.transfer_learning import plot_history, check_accuracy


2022-01-26 11:17:37.512254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-26 11:17:37.512380: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Found 1694 images belonging to 11 classes.
Found 506 images belonging to 11 classes.


In [6]:
model_test = compile_model()

2022-01-26 11:18:51.587602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-26 11:18:51.588024: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-26 11:18:51.588150: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-6U135SI): /proc/driver/nvidia/version does not exist
2022-01-26 11:18:51.589730: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
history = train_model(model_test,name_to_save='model_test')


In [9]:
check_accuracy(pick_model='model_test')


TITLE: Colias Croceus
PREDICTIONS: {'Colias Croceus': 1.0}
BEST PREDICTION: Colias Croceus
RESULT Good prediction!


TITLE: Maniola Jurtina
PREDICTIONS: {'Maniola Jurtina': 1.0}
BEST PREDICTION: Maniola Jurtina
RESULT Good prediction!


TITLE: Iphiclides Feisthamelii
PREDICTIONS: {'Iphiclides Feisthamelii': 1.0}
BEST PREDICTION: Iphiclides Feisthamelii
RESULT Good prediction!


TITLE: Aricia Cramera
PREDICTIONS: {'Aricia Cramera': 1.0}
BEST PREDICTION: Aricia Cramera
RESULT Good prediction!


TITLE: Coenonympha Pamphilus
PREDICTIONS: {'Coenonympha Pamphilus': 1.0}
BEST PREDICTION: Coenonympha Pamphilus
RESULT Good prediction!


TITLE: Maniola Jurtina
PREDICTIONS: {'Coenonympha Pamphilus': 1.0}
BEST PREDICTION: Coenonympha Pamphilus
RESULT Wrong Prediction!


TITLE: Lysandra Bellargus
PREDICTIONS: {'Lysandra Bellargus': 1.0}
BEST PREDICTION: Lysandra Bellargus
RESULT Good prediction!


TITLE: Issoria Lathonia
PREDICTIONS: {'Issoria Lathonia': 1.0}
BEST PREDICTION: Issoria Lathonia
RESUL